## Day 34 Lecture 2 Assignment

In this assignment, we will learn about gradient boosting. We will use a dataset describing TripAdvisor reviews for Las Vegas hotels loaded below and analyze the model generated for this dataset.

In [1]:
import numpy as np
import pandas as pd

import timeit
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

from category_encoders import LeaveOneOutEncoder
from sklearn.pipeline import Pipeline



import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
vegas = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/LasVegasTripAdvisorReviews-Dataset.csv', sep=';')

In [3]:
vegas.head()

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,USA,11,4,13,5,Dec-Feb,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,9,January,Thursday
1,USA,119,21,75,3,Dec-Feb,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,3,January,Friday
2,USA,36,9,25,5,Mar-May,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,2,February,Saturday
3,UK,14,7,14,4,Mar-May,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,Europe,6,February,Friday
4,Canada,5,5,2,4,Mar-May,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3,3773,North America,7,March,Tuesday


Check for missing data and remove all rows with missing data

In [4]:
# answer below: 
vegas.isna().mean()


User country         0.0
Nr. reviews          0.0
Nr. hotel reviews    0.0
Helpful votes        0.0
Score                0.0
Period of stay       0.0
Traveler type        0.0
Pool                 0.0
Gym                  0.0
Tennis court         0.0
Spa                  0.0
Casino               0.0
Free internet        0.0
Hotel name           0.0
Hotel stars          0.0
Nr. rooms            0.0
User continent       0.0
Member years         0.0
Review month         0.0
Review weekday       0.0
dtype: float64

In [5]:
vegas.sort_values(by = ['Hotel name'])

,User country,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
455,Malaysia,20,18,19,2,Dec-Feb,Families,YES,YES,NO,YES,YES,YES,Bellagio Las Vegas,5,3933,Asia,4,December,Friday
433,USA,15,3,13,5,Dec-Feb,Families,YES,YES,NO,YES,YES,YES,Bellagio Las Vegas,5,3933,North America,5,January,Thursday
434,USA,30,6,26,4,Dec-Feb,Couples,YES,YES,NO,YES,YES,YES,Bellagio Las Vegas,5,3933,North America,3,February,Tuesday
435,USA,41,9,111,5,Dec-Feb,Business,YES,YES,NO,YES,YES,YES,Bellagio Las Vegas,5,3933,North America,1,February,Friday
436,India,44,35,53,5,Mar-May,Couples,YES,YES,NO,YES,YES,YES,Bellagio Las Vegas,5,3933,Asia,6,March,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,USA,164,48,82,5,Dec-Feb,Couples,YES,YES,YES,YES,YES,YES,Wynn Las Vegas,5,2700,North America,4,January,Saturday
193,USA,5,3,9,5,Dec-Feb,Families,YES,YES,YES,YES,YES,YES,Wynn Las Vegas,5,2700,North America,4,January,Tuesday
194,Canada,7,5,20,4,Dec-Feb,Couples,YES,YES,YES,YES,YES,YES,Wynn Las Vegas,5,2700,North America,3,February,Saturday
203,USA,5,4,4,5,Jun-Aug,Couples,YES,YES,YES,YES,YES,YES,Wynn Las Vegas,5,2700,North America,11,June,Wednesday


How many reviews do we have for each hotel in the dataset?

In [6]:
vegas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504 entries, 0 to 503
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   User country       504 non-null    object
 1   Nr. reviews        504 non-null    int64 
 2   Nr. hotel reviews  504 non-null    int64 
 3   Helpful votes      504 non-null    int64 
 4   Score              504 non-null    int64 
 5   Period of stay     504 non-null    object
 6   Traveler type      504 non-null    object
 7   Pool               504 non-null    object
 8   Gym                504 non-null    object
 9   Tennis court       504 non-null    object
 10  Spa                504 non-null    object
 11  Casino             504 non-null    object
 12  Free internet      504 non-null    object
 13  Hotel name         504 non-null    object
 14  Hotel stars        504 non-null    object
 15  Nr. rooms          504 non-null    int64 
 16  User continent     504 non-null    object
 1

In [7]:
vegas = vegas.drop(columns=['User country', 'Period of stay', 'Period of stay', 'Review weekday', 'Hotel stars' ])

In [8]:
cat_col = list(vegas.columns[vegas.dtypes == 'object'])
num_cols =['Nr. reviews',
 'Nr. hotel reviews',
 'Helpful votes',
 'Nr. rooms',
 'Member years']

In [9]:
vegas['Traveler type'].value_counts()

Couples     214
Families    110
Friends      82
Business     74
Solo         24
Name: Traveler type, dtype: int64

In [10]:
# answer below:

We would like to predict the score variable. Examine the dataset and decide which columns should be turned into dummy variables and transform the data. Also, where we have two columns with redundant information, remove one of the two columns. Remove the hotel stars column.

In [11]:
# answer below:
vegas.corr()['Score'].sort_values(ascending=False).abs()

Score                1.000000
Helpful votes        0.015472
Nr. hotel reviews    0.013718
Nr. reviews          0.024320
Member years         0.036190
Nr. rooms            0.080200
Name: Score, dtype: float64

Split the data into train and test (20% in test)

In [12]:
vegas

,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Nr. rooms,User continent,Member years,Review month
0,11,4,13,5,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3773,North America,9,January
1,119,21,75,3,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3773,North America,3,January
2,36,9,25,5,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3773,North America,2,February
3,14,7,14,4,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3773,Europe,6,February
4,5,5,2,4,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3773,North America,7,March
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,15,4,8,5,Couples,YES,YES,NO,YES,YES,YES,The Westin las Vegas Hotel Casino & Spa,826,Europe,1,October
500,50,13,29,4,Couples,YES,YES,NO,YES,YES,YES,The Westin las Vegas Hotel Casino & Spa,826,North America,8,November
501,154,23,31,4,Friends,YES,YES,NO,YES,YES,YES,The Westin las Vegas Hotel Casino & Spa,826,North America,4,November
502,9,6,5,2,Families,YES,YES,NO,YES,YES,YES,The Westin las Vegas Hotel Casino & Spa,826,North America,9,December


In [13]:
# answer below:
X = vegas.drop(columns=['Score'])
y = vegas['Score']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [14]:
X


,Nr. reviews,Nr. hotel reviews,Helpful votes,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Nr. rooms,User continent,Member years,Review month
0,11,4,13,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3773,North America,9,January
1,119,21,75,Business,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3773,North America,3,January
2,36,9,25,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3773,North America,2,February
3,14,7,14,Friends,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3773,Europe,6,February
4,5,5,2,Solo,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3773,North America,7,March
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,15,4,8,Couples,YES,YES,NO,YES,YES,YES,The Westin las Vegas Hotel Casino & Spa,826,Europe,1,October
500,50,13,29,Couples,YES,YES,NO,YES,YES,YES,The Westin las Vegas Hotel Casino & Spa,826,North America,8,November
501,154,23,31,Friends,YES,YES,NO,YES,YES,YES,The Westin las Vegas Hotel Casino & Spa,826,North America,4,November
502,9,6,5,Families,YES,YES,NO,YES,YES,YES,The Westin las Vegas Hotel Casino & Spa,826,North America,9,December


In [15]:
y

0      5
1      3
2      5
3      4
4      4
      ..
499    5
500    4
501    4
502    2
503    4
Name: Score, Length: 504, dtype: int64

In [16]:
start_time = timeit.default_timer()

preprocessing = ColumnTransformer([
    ('encode_cats', OneHotEncoder(), cat_col),
    ('scale_nums', StandardScaler(), num_cols)             
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ("gbr", GradientBoostingRegressor())

])

grid = {
    "gbr__subsample": [1.0],
    "gbr__max_features": [0.5],
    "gbr__max_depth": [7],
}
pipeline_gl = GridSearchCV(pipeline, grid, verbose = 1)
pipeline.fit(X_train, y_train)
# pipeline_gl.best_params_
train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

# elapsed = timeit.default_timer() - start_time
# print(elapsed)

In [21]:
X_train

,Nr. reviews,Nr. hotel reviews,Helpful votes,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Nr. rooms,User continent,Member years,Review month
303,48,21,19,Families,YES,YES,NO,YES,YES,YES,Hilton Grand Vacations on the Boulevard,1228,Europe,4,August
198,125,35,48,Business,YES,YES,YES,YES,YES,YES,Wynn Las Vegas,2700,North America,4,April
78,15,14,7,Friends,YES,YES,YES,YES,YES,YES,Treasure Island- TI Hotel & Casino,2884,North America,9,April
75,17,9,16,Solo,YES,YES,YES,YES,YES,YES,Treasure Island- TI Hotel & Casino,2884,North America,-1806,February
391,16,4,7,Couples,YES,YES,YES,NO,NO,YES,Wyndham Grand Desert,787,North America,4,April
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,17,10,39,Friends,YES,YES,NO,YES,YES,YES,Bellagio Las Vegas,3933,North America,3,November
61,9,6,19,Couples,YES,YES,NO,YES,YES,NO,Monte Carlo Resort&Casino,3003,Oceania,7,July
9,12,7,11,Families,NO,YES,NO,NO,YES,YES,Circus Circus Hotel & Casino Las Vegas,3773,North America,5,May
418,7,5,6,Couples,YES,YES,NO,YES,YES,YES,The Venetian Las Vegas Hotel,4027,North America,1,June


Create a graident boosted regression model for predicting the score. To produce the accuracy score for the test data, first round the prediction and then compare to the observed.

In [17]:
# answer below:
print(f"train_score {train_score}")
print(f"test_score {test_score}")


train_score 0.64205144342532
test_score 0.11063308468895394


Try again with a learning rate of 0.8 and 0.3 and compare the results.

In [37]:
# Adjusted max_features/max_depth to have smaller grid
grid = {
    "gbr__subsample": [1.0],
    "gbr__max_features": [0.5],
    "gbr__max_depth": [9],
}

n_trees = 100
learning_rate = .8

preprocessing = ColumnTransformer([
    ('encode_cats', OneHotEncoder(), cat_col),
    ('scale_nums', StandardScaler(), num_cols)             
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ("gbr", GradientBoostingRegressor( 
                                      learning_rate=learning_rate))

])
pipeline_cv.best_params_
pipeline_gl = GridSearchCV(pipeline, grid, verbose = 1)
pipeline_gl.fit(X_train, y_train)
#pipeline_gl.best_params_
train_score = pipeline_gl.score(X_train, y_train)
test_score = pipeline_gl.score(X_test, y_test)

# elapsed = timeit.default_timer() - start_time
# print(elapsed)
print(f"train_score {train_score}")
print(f"test_score {test_score}")

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


train_score 0.9999999132849461
test_score -0.45549333998973607


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.2s finished


In [60]:
# Adjusted max_features/max_depth to have smaller grid
grid = {
    "gbr__subsample": [1.0],
    "gbr__max_features": [0.5],
    "gbr__max_depth": [9],
}

learning_rate = .3

preprocessing = ColumnTransformer([
    ('encode_cats', OneHotEncoder(), cat_col),
    ('scale_nums', StandardScaler(), num_cols)             
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessing', preprocessing),
    ("gbr", GradientBoostingRegressor(learning_rate=.3, max_depth = 100, n_iter_no_change = 1000, n_estimators = 10000))

])
#pipeline_cv.best_params_
#pipeline_gl = GridSearchCV(pipeline, grid, verbose = 1)
pipeline.fit(X_train, y_train)

train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

# elapsed = timeit.default_timer() - start_time
# print(elapsed)
print(f"train_score {train_score}")
print(f"test_score {test_score}")

train_score 0.8275648438242116
test_score -0.8193421288431231


In [50]:

train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

# elapsed = timeit.default_timer() - start_time
# print(elapsed)
print(f"train_score {train_score}")
print(f"test_score {test_score}")

train_score 0.9999999019143667
test_score -0.4393991581395771
